In [0]:
!wget https://learner.csie.ntu.edu.tw/~judge/ml19spring/ml19spring.zip
!unzip ml19spring.zip

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

prefix = '/content/gdrive/My Drive/'

In [0]:
!rm ml19spring.zip

In [0]:
!unzip /content/gdrive/My\ Drive/colab.zip -d ./

In [0]:
import sys
sys.path.insert(0, './colab/')
from training_utils import *

In [0]:
# specify id
y_id = 2
track_id = 1

In [21]:
from xgboost import XGBRegressor
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

test_x = np.load('X_test.npz')['arr_0']
train_x = np.load('X_train.npz')['arr_0']
train_y = np.load('Y_train.npz')['arr_0'][:, y_id]
all_x = train_x.copy()
all_y = train_y.copy()
print(test_x.shape, train_x.shape, train_y.shape)

(2500, 10000) (47500, 10000) (47500,)


In [22]:
# pick only important data
idx = {}
with open('colab/29/adaboost' + str(y_id) + '_feature.csv', 'r') as f:
    i = 0
    for lines in f:
        importance = float(lines.replace('\n', '').split(',')[y_id])
        if i not in idx:
            idx[i] = 0
        idx[i] += importance
        i += 1
with open('colab/28/adaboost' + str(y_id) + '_feature.csv', 'r') as f:
    i = 0
    for lines in f:
        importance = float(lines.replace('\n', '').split(',')[y_id])
        if i not in idx:
            idx[i] = 0
        idx[i] += importance
        i += 1

with open('colab/32/adaboost' + str(y_id) + '_feature.csv', 'r') as f:
    i = 0
    for lines in f:
        importance = float(lines.replace('\n', '').split(',')[y_id])
        if i not in idx:
            idx[i] = 0
        idx[i] += importance
        i += 1
with open('colab/35/random_forest' + str(y_id) + '_feature.csv', 'r') as f:
    i = 0
    for lines in f:
        importance = float(lines.replace('\n', '').split(',')[y_id])
        if i not in idx:
            idx[i] = 0
        idx[i] += importance
        i += 1
    
idxx = [i[0] for i in idx.items() if i[1] > 1e-3]
print(len(idxx))
train_x = train_x[:, idxx]
test_x = test_x[:, idxx]
all_x = all_x[:, idxx]
print(train_x.shape)

229
(47500, 229)


In [23]:
from sklearn.model_selection import train_test_split

train_x, val_x, train_y, val_y = train_test_split(train_x, train_y, test_size=0.052631578947368, random_state=1126)
print(train_x.shape, val_x.shape, train_y.shape, val_y.shape)

(45000, 229) (2500, 229) (45000,) (2500,)


In [0]:
# define my own scorer
# actually it should be called error function here
def scorer(y_pred, y_true):
    return 'error', 1.0 * np.sum(np.abs(y_true.get_label() - y_pred)) / len(y_pred)
  
# define my own error function
def mae(y_true, y_pred):
    grad = np.tanh(y_pred - y_true)
    hess = 1 - grad * grad
#    print(y_pred, y_true)
#    print(y_pred - y_true)
#    print(grad)
    return grad, hess
  
params = {
    'objective': mae,
    'max_depth': 9,
    'learning_rate': 0.1,
    'verbosity': 2,
    'tree_method': 'gpu_hist',
    'predictor': 'gpu_predictor',
    'n_estimators': 500,
    'n_jobs': 1,
    'subsample': 0.75,
    'booster': 'dart'
}



In [25]:
from xgboost import XGBRegressor
model = XGBRegressor(**params)
model.fit(train_x, train_y, eval_set=[(val_x, val_y)], eval_metric=scorer, early_stopping_rounds=None)

[0]	validation_0-rmse:0.25478	validation_0-error:0.22132
[1]	validation_0-rmse:0.227432	validation_0-error:0.197829
[2]	validation_0-rmse:0.203449	validation_0-error:0.177115
[3]	validation_0-rmse:0.182262	validation_0-error:0.158773
[4]	validation_0-rmse:0.163462	validation_0-error:0.142452
[5]	validation_0-rmse:0.146752	validation_0-error:0.12793
[6]	validation_0-rmse:0.131837	validation_0-error:0.114944
[7]	validation_0-rmse:0.118533	validation_0-error:0.103359
[8]	validation_0-rmse:0.106643	validation_0-error:0.092989
[9]	validation_0-rmse:0.096015	validation_0-error:0.083724
[10]	validation_0-rmse:0.086483	validation_0-error:0.075395
[11]	validation_0-rmse:0.077918	validation_0-error:0.067909
[12]	validation_0-rmse:0.070249	validation_0-error:0.061194
[13]	validation_0-rmse:0.063354	validation_0-error:0.055155
[14]	validation_0-rmse:0.057186	validation_0-error:0.049755
[15]	validation_0-rmse:0.051661	validation_0-error:0.044912
[16]	validation_0-rmse:0.046719	validation_0-error:0.

XGBRegressor(base_score=0.5, booster='dart', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, importance_type='gain',
       learning_rate=0.1, max_delta_step=0, max_depth=9,
       min_child_weight=1, missing=None, n_estimators=500, n_jobs=1,
       nthread=None, objective=<function mae at 0x7f94480d29d8>,
       predictor='gpu_predictor', random_state=0, reg_alpha=0,
       reg_lambda=1, scale_pos_weight=1, seed=None, silent=True,
       subsample=0.75, tree_method='gpu_hist', verbosity=2)

In [0]:
model.save_model(prefix + '46tmp.model')

In [26]:
#from xgboost import XGBRegressor
#model = XGBRegressor(**params)
#model.load_model(prefix + '2_46.model')

print("ein1:", err1_calc(model.predict(train_x), train_y, y_id))
print("eval1:", err1_calc(model.predict(val_x), val_y, y_id))
print("eall1:", err1_calc(model.predict(all_x), all_y, y_id))

print("ein2:", err2_calc(model.predict(train_x), train_y))
print("eval2:", err2_calc(model.predict(val_x), val_y))
print("eall2:", err2_calc(model.predict(all_x), all_y))



ein1: 0.3623483794978159
eval1: 1.7800182888665266
eall1: 0.43696258525406384
ein2: 0.0016607235415628963
eval2: 0.008282795016307319
eall2: 0.0020092536191810236


In [0]:
# write files
write_prediction(prefix + '47_train_y' + str(y_id) + '_' + str(track_id) + '.txt', 'w', model.predict(all_x).reshape((47500, 1)).astype('str'))
write_prediction(prefix + '47_test_y' + str(y_id) + '_' + str(track_id) + '.txt', 'w', model.predict(test_x).reshape((2500, 1)).astype('str'))
model.save_model(prefix + str(y_id) + '_47.model')

In [11]:
model2 = XGBRegressor(**params)
model2.load_model(prefix + '46tmp.model')

print("ein1:", err1_calc(model2.predict(train_x), train_y, y_id))
print("eval1:", err1_calc(model2.predict(val_x), val_y, y_id))
print("eall1:", err1_calc(model2.predict(all_x), all_y, y_id))

print("ein2:", err2_calc(model2.predict(train_x), train_y))
print("eval2:", err2_calc(model2.predict(val_x), val_y))
print("eall2:", err2_calc(model2.predict(all_x), all_y))


ein1: 11.1401772240326
eval1: 18.824184339254384
eall1: 11.544598651149535
ein2: 0.0909864887588319
eval2: 0.1511424652330306
eall2: 0.09415259278378975
